#### Signal analysis
- 1ks plot (6x subplots) Mafaulda welch from each fault (1s, 2**14 window, hann window)  - largest severity - 2500 rpm
- 1ks plot (6x subplots) Custom dataset - each place in one day spectrum (5s segment)

In [ ]:
import os
from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mafaulda
import extraction
import pumps

PATH = '../datasets/'
MAFAULDA_PATH = os.path.join(PATH, 'MAFAULDA.zip')
PUMPS_PATH = os.path.join(PATH, 'FluidPump.zip')
FEATURES_PATH = os.path.join(PATH, 'features')
KSB_CLOUD_PATH = os.path.join(PATH, 'ksb-cloud/pumps-compare')

FFT_WINDOW = 2 ** 15

In [ ]:
# Mafaulda - worst severity at 2500 rpm
def plot_bearing_frequencies(freqs, pxx, bearings, ax, name, deltaF):
    position = {
        'M1-1': 'MTR001',
        'M2-1': 'MTR001',
        'M1-2': 'MTR002',
        'M2-2': 'MTR002',
        'P1-3': 'PMP003',
        'P2-3': 'PMP003',
        'P1-4': 'PMP004',
        'P2-4': 'PMP004',
    }
    bearing = bearings[
        bearings['placement'] == position.get(name, name)
    ]
    columns = ['RPM', 'BPFO', 'BPFI', 'BSF', 'FTF']
    colors = ['red', 'purple', 'orange', 'black', 'green']
    markers = ['s', 'o', 'D', 'P', 'x']
    n_harmonics = 20

    for col, color, mark in zip(columns, colors, markers):
        f_fundamental = bearing[col].values[0]
        f_harmonics = [i * f_fundamental for i in range(1, n_harmonics+1)]
        amplitudes = [
            pxx[int(f/deltaF)] for f in f_harmonics 
            if int(f/deltaF) < len(pxx)
        ]
        f_harmonics = f_harmonics[:len(amplitudes)]
        ax.plot(
            f_harmonics,
            amplitudes,
            color=color,
            marker=mark,
            markerfacecolor='None',
            markeredgecolor=color,
            linestyle='None',
            label=col,
            markeredgewidth=2
        )

    return ax.get_legend_handles_labels()

def plot_psd(
        ts, axname, fs, name, ax, window=FFT_WINDOW, 
        xlim=None, ylim=None, dB=False, bearings=None, 
        freqs=None, pxx=None):

    if freqs is None or pxx is None:
        freqs, pxx = extraction.spectral_transform(ts[axname], window, fs)

    if dB is True:
        pxx = 20 * np.log10(pxx / 0.000001)
        ax.set_ylabel('Amplitude [dB]')
    else:
        ax.set_ylabel('Amplitude [m/s\u00B2]')
        

    ax.plot(freqs, pxx, color='darkblue')
    ax.fill_between(freqs, pxx, color='lightblue', alpha=0.3)

    legend = None
    if bearings is not None:
        legend = plot_bearing_frequencies(freqs, pxx, bearings, ax, name, fs / window)

    ax.grid(True)
    #ax.set_xlabel('Frequency [Hz]')

    if xlim is not None:
        ax.set_xlim(0, xlim)
    if ylim is not None:
        ax.set_ylim(0, ylim)
    ax.set_title(name)

    return legend

In [ ]:
axname = 'ay'
dataset = ZipFile(MAFAULDA_PATH)
filenames = {
    'normal/43.6224.csv': 'normal',
    'horizontal-misalignment/2.0mm/42.5984.csv': 'misalignment',
    'imbalance/35g/43.6224.csv': 'imbalance',
    'underhang/cage_fault/35g/43.4176.csv': 'cage fault',
    'underhang/ball_fault/35g/41.1648.csv': 'ball fault',
    'underhang/outer_race/35g/43.4176.csv': 'outer race fault'
}
fig, ax = plt.subplots(len(filenames), 1, figsize=(10, 12), sharex=True)
for i, name in enumerate(filenames.items(), start=0):
    filename, title = name
    ts = mafaulda.csv_import(dataset, filename)
    plot_psd(ts, axname, mafaulda.SAMPLING_RATE, title, ax[i], xlim=2500, ylim=0.03)
ax[-1].set_xlabel('Frequency [Hz]')

fig.tight_layout()
plt.show()

In [ ]:
# Show Mafaulda faults
# Detail on bearing frequencies
fig, ax = plt.subplots(len(filenames), 1, figsize=(10, 15), sharex=True)
for i, name in enumerate(filenames.items(), start=0):
    filename, title = name
    ts = mafaulda.csv_import(dataset, filename)
    rpm = ts['rpm'].mean()
    machine = mafaulda.bearing_frequencies(rpm)
    machine['placement'] = title
    machine = pd.DataFrame.from_records([machine])

    handles, labels = plot_psd(
        ts, axname, mafaulda.SAMPLING_RATE, title, ax[i],
        xlim=1000, window=2**15, bearings=machine, dB=False
    )
    ax[i].set_title(f'{title} ({rpm:.0f} rpm)')
    ax[i].set_ylim(0, 0.02)
ax[-1].set_xlabel('Frequency [Hz]')

lines = 5
fig.legend(handles, labels, loc='lower center', ncol=lines, numpoints=1)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
axname = 'z'
dataset = ZipFile(PUMPS_PATH)
filenames = {
    '2024-02-20/K3/001/1.tsv': 'C1-1',
    '2024-02-20/K5/001/1.tsv': 'C2-1',
    '2024-02-20/K3/002/1.tsv': 'C1-2',
    '2024-02-20/K5/002/1.tsv': 'C2-2'
}

fig, ax = plt.subplots(len(filenames), 1, figsize=(8, 10))
for i, name in enumerate(filenames.items(), start=0):
    filename, title = name
    ts = pumps.csv_import(dataset, filename)
    plot_psd(ts, axname, pumps.SAMPLING_RATE, title, ax[i], xlim=2000, ylim=2)

fig.tight_layout()
plt.show()

In [ ]:
# Bearing frequencies
BEARINGS_FILE = os.path.join(FEATURES_PATH, 'bearings.csv')
BEARINGS_PROCESSED_FILE = os.path.join(FEATURES_PATH, 'bearings_processed.csv')

# http://webtools3.skf.com/engcalc/CalcBearingFrequencies.do
# https://www.skfbearingselect.com/#/type-arrangement/single-bearing
bearings = pd.read_csv(BEARINGS_FILE)
bearings['RPM'] = bearings['rpm'] / 60
# Outer race
bearings['BPFO'] = (
    (bearings['balls'] / 2) *
    (bearings['rpm'] / 60) *
    (1 - (bearings['ball_diameter'] / bearings['pitch_diameter']) * np.cos(np.radians(bearings['angle'])))
)
# Inner race
bearings['BPFI'] = (
    (bearings['balls'] / 2) *
    (bearings['rpm'] / 60) *
    (1 + (bearings['ball_diameter'] / bearings['pitch_diameter']) * np.cos(np.radians(bearings['angle'])))
)
# Ball
bearings['BSF'] = (
    (bearings['pitch_diameter'] / (2 * bearings['ball_diameter'])) *
    (bearings['rpm'] / 60) *
    (1 + ((bearings['ball_diameter'] / bearings['pitch_diameter']) * np.cos(np.radians(bearings['angle'])))**2)
)
# Cage
bearings['FTF'] = (
    0.5 * (bearings['rpm'] / 60) *
    (1 - (bearings['ball_diameter'] / bearings['pitch_diameter']) * np.cos(np.radians(bearings['angle'])))
)
bearings.to_csv(BEARINGS_PROCESSED_FILE)
bearings

In [ ]:
axname = 'z'
dataset = ZipFile(PUMPS_PATH)
i = 1
date = '2024-03-26'
filenames = {
    f'{date}/KSB1/MTR001/{i}.tsv': 'M1-1',
    f'{date}/KSB7/MTR001/{i}.tsv': 'M2-1',
    f'{date}/KSB1/MTR002/{i}.tsv': 'M1-2',
    f'{date}/KSB7/MTR002/{i}.tsv': 'M2-2'
}

fig, ax = plt.subplots(len(filenames), 1, figsize=(10, 15), sharex=True)
for i, name in enumerate(filenames.items(), start=0):
    filename, title = name
    ts = pumps.csv_import(dataset, filename)
    plot_psd(ts, axname, pumps.SAMPLING_RATE, title, ax[i], xlim=6000, ylim=0.2)
ax[-1].set_xlabel('Frequency [Hz]')
fig.tight_layout()
plt.show()

In [ ]:
# Detail on bearing frequencies
fig, ax = plt.subplots(len(filenames), 1, figsize=(10, 15), sharex=True)
for i, name in enumerate(filenames.items(), start=0):
    filename, title = name
    ts = pumps.csv_import(dataset, filename)
    handles, labels = plot_psd(ts, axname, pumps.SAMPLING_RATE, title, ax[i], xlim=1000, dB=False, window=2**16, bearings=bearings)
    ax[i].set_ylim(0, 0.02)

ax[-1].set_xlabel('Frequency [Hz]')

lines = 5
fig.legend(handles, labels, loc='lower center', ncol=lines, numpoints=1)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
axname = 'z'
dataset = ZipFile(PUMPS_PATH)
i = 1
date = '2024-02-27'
filenames = {
    f'{date}/KSB1/PMP003/{i}.tsv': 'P1-3',
    f'{date}/KSB7/PMP003/{i}.tsv': 'P2-3',
    f'{date}/KSB1/PMP004/{i}.tsv': 'P1-4',
    f'{date}/KSB7/PMP004/{i}.tsv': 'P2-4'
}

# Overall
fig, ax = plt.subplots(len(filenames), 1, figsize=(10, 15),  sharex=True)
for i, name in enumerate(filenames.items(), start=0):
    filename, title = name
    ts = pumps.csv_import(dataset, filename)
    plot_psd(ts, axname, pumps.SAMPLING_RATE, title, ax[i], xlim=6000, ylim=0.1)

ax[-1].set_xlabel('Frequency [Hz]')
fig.tight_layout()
plt.show()

In [ ]:
# Detail on bearing frequencies
fig, ax = plt.subplots(len(filenames), 1, figsize=(10, 15), sharex=True)
for i, name in enumerate(filenames.items(), start=0):
    filename, title = name
    ts = pumps.csv_import(dataset, filename)
    handles, labels = plot_psd(ts, axname, pumps.SAMPLING_RATE, title, ax[i], xlim=1000, dB=False, window=2**16, bearings=bearings)
    ax[i].set_ylim(0, 0.06)

ax[-1].set_xlabel('Frequency [Hz]')

lines = 5
fig.legend(handles, labels, loc='lower center', ncol=lines, numpoints=1)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
axname = 'z'
i = 1
filenames = {
    'ksb-1-27-2-6-27.csv': 'P1-3',
    'ksb-7-28-2-11-43.csv': 'P2-3',
    'ksb-1-fft-26-3-6-33.csv': 'P1-3',
    'ksb-7-fft-13-3-5-33.csv': 'P2-3' 
}
fs_ksb = 2000

fig, ax = plt.subplots(len(filenames), 1, figsize=(10, 15))
for i, name in enumerate(filenames.items(), start=0):
    filename, title = name

    ts = pd.read_csv(
        os.path.join(KSB_CLOUD_PATH, filename),
        delimiter=';',
        decimal=',',
        index_col=False
    )
    ts.rename(columns={
        'Frequency [Hertz]': 'f', 
        'AmplitudeX [mm/s]': 'x',
        'AmplitudeY [mm/s]': 'y',
        'AmplitudeZ [mm/s]': 'z'
    }, inplace=True)
    ts = ts.set_index('f')
    
    handles, labels = plot_psd(
        None, axname, fs_ksb, title, ax[i], 
        dB=False, window=512, xlim=1000,
        bearings=bearings,
        freqs=ts.index.to_numpy(), pxx=ts['z'].to_numpy())

lines = 5
fig.legend(handles, labels, loc='lower center', ncol=lines)
fig.tight_layout()
plt.show()